In [106]:
import pandas as pd
import os

In [2]:
PATH = "../data_given/"

In [3]:
df = pd.read_csv(PATH + "winequality.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,TARGET
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,TARGET
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [8]:
overview = df.describe()
overview.loc[["min", "max"]].to_json("schema.json")

In [11]:
class NotInRange(Exception):
    def __init__(self, message="value not in range"):
        #self.input_ = input_
        self.message = message
        super().__init__(self.message)

In [12]:
raise NotInRange

NotInRange: value not in range

In [35]:
import numpy as np
x = [['1'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'], ['10']]

In [36]:
x = [i[0] for i in x]
x = list(map(float, x))
x = np.array(x)

In [40]:
x.reshape(-1,1)

array([[ 1.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 7.],
       [ 8.],
       [ 9.],
       [10.]])

In [37]:
import joblib

model = joblib.load('../saved_models/model.joblib')

/Users/ashrafkhan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator ElasticNet from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [49]:
sample = df.drop('TARGET', axis=1)
sample.values[0]

array([ 7.4   ,  0.7   ,  0.    ,  1.9   ,  0.076 , 11.    , 34.    ,
        0.9978,  3.51  ,  0.56  ,  9.4   ])

In [118]:
model.predict([sample.values[0]])

array([5.67957138])

In [133]:
import mlflow
from mlflow.tracking import MlflowClient
import argparse
from pprint import pprint
import pickle, joblib

def log_production_model(config):
    mlflow_config = config["mlflow_config"]
    remote_server_uri = mlflow_config["remote_server_uri"]
    registered_model_name = mlflow_config["registered_model_name"]

    mlflow.set_tracking_uri(remote_server_uri)

    runs = mlflow.search_runs(experiment_ids=1)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name = '{registered_model_name}'"):
        mv = dict(mv)
        print(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=registered_model_name,
                version=current_version,
                stage="Production"
            )
        else:
            # If some model already present in Production
            current_version = mv["version"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=registered_model_name,
                version=current_version,
                stage="Staging"
            )
    model_path = os.path.join(logged_model, "model.pkl")
    
    with open(model_path, "rb") as f:
        model = joblib.load(f)
    
    saved_model_path = os.path.join(config["webapp_model_dir"], "model.joblib")
    print(saved_model_path)
    joblib.dump(model, saved_model_path)    

In [54]:
import yaml


def get_config(config_path="../params.yaml"):
    with open(config_path) as yaml_file:
        config = yaml.safe_load(yaml_file)
    return config

config = get_config()

In [57]:
mlflow_config = config["mlflow_config"]

In [134]:
log_production_model(config)

{'creation_timestamp': 1642158044793, 'current_stage': 'Staging', 'description': '', 'last_updated_timestamp': 1642319504111, 'name': 'ElasticNetWine', 'run_id': 'f237a54f69f64a93a09e04abf3a8eb2c', 'run_link': '', 'source': './artifacts/1/f237a54f69f64a93a09e04abf3a8eb2c/artifacts/model', 'status': 'READY', 'status_message': '', 'tags': {}, 'user_id': '', 'version': '1'}
{   'creation_timestamp': 1642158044793,
    'current_stage': 'Staging',
    'description': '',
    'last_updated_timestamp': 1642319504111,
    'name': 'ElasticNetWine',
    'run_id': 'f237a54f69f64a93a09e04abf3a8eb2c',
    'run_link': '',
    'source': './artifacts/1/f237a54f69f64a93a09e04abf3a8eb2c/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}
{'creation_timestamp': 1642158136768, 'current_stage': 'Staging', 'description': '', 'last_updated_timestamp': 1642319504129, 'name': 'ElasticNetWine', 'run_id': 'd3d56d5b15444b9bafb9d8955df15d8c', 'ru

/Users/ashrafkhan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator ElasticNet from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [126]:
pwd

'/Users/ashrafkhan/Desktop/Projects/Wine2'

In [99]:
x

ElasticNet(alpha=0.7, l1_ratio=0.78, normalize='deprecated', random_state=42)

In [135]:
saved_model_path = config["webapp_model_dir"] + "/model.joblib"
joblib.load(saved_model_path)

ElasticNet(alpha=0.7, l1_ratio=0.78, normalize='deprecated', random_state=42)